In [1]:
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import numpy as np
import pandas as pd
import sobol_seq
import matplotlib.pyplot as plt
from functools import partial


In [2]:
Genome = List[int]
Population = List[Genome]
FitnessFunc = Callable[[Genome], float]
PopulateFunc = Callable[[], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]

Schewel

In [3]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         sum_sq = 0
#         for j in range (i+1):
#             sum_sq += x[j]
#         f += sum_sq**2 
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

2n Minima - Error

In [4]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += (x[i]**4-16*x[i]**2+5*x[i])
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

Rastrigin

In [5]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += (x[i]**2-10*np.cos(2*np.pi*x[i])+10)
#     return f
# boundaries = np.array([(-5,5) for _ in range (dim)])

Griewank

In [6]:
# n_point = 100
# k_max = 100
# dim = 3
# epsilon = 10**(-7)
# num_bits = 64
# def objective_function(x):
#     f=0
#     for i in range (len(x)):
#         f += x[i]**2
#     f = f/4000
#     for i in range (len(x)):
#         f *= np.cos(x[i]/np.sqrt(i))
#     return f
# boundaries = np.array([(-50,50) for _ in range (dim)])

Problem 1 - Paper Pak Kun

In [7]:
# n_point = 100
# k_max = 250
# dim = 2
# epsilon = 10**(-7)
# num_bits = 64

# def objective_function(x):
#     f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
#     f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
#     f_list = [f1,f2]
#     denom = 0
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/(1+denom)
#     return F
# boundaries = np.array([(-10,10) for _ in range (dim)])

Problem 1 - Paper Nikos

In [8]:
n_point = 100
k_max = 20
dim = 2
epsilon = 10**(-7)
num_bits = 64
p_mutation = 0.1

def objective_function(x):
    # F = (x[0]**2+x[0]*x[1]-6)**2+(x[0]**2+x[1]**3+2*x[0]*x[1]**2-3)**2
    F = abs(x[0]**2+x[0]*x[1]-6)+abs(x[0]**2+x[1]**3+2*x[0]*x[1]**2-3)
    # F = x[0]**2+x[1]**2
    return F
boundaries = np.array([(-100,100) for _ in range (dim)])

Saddle Note Bifurcation

In [9]:
# n_point = 20
# k_max = 250
# dim = 2
# epsilon = 10**(-7)
# num_bits = 64

# def objective_function(X):
#     f1 = X[0]+X[1]**2 #-2*x
#     f_list = [f1]
#     denom = 1
#     for f in f_list:
#         denom +=np.abs(f)
#     F = 1/denom
#     return F

# boundaries = np.array([(-10,10) for _ in range (dim)])

In [10]:
min_value = boundaries.min()
max_value = boundaries.max()
# num_bits = 64  # Number of bits for each number
print(min_value,max_value)

-100 100


In [11]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim,npoint,low=-10,high=10):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    # Generate Sobol sequence points
    sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)

    # Scale the Sobol points to fit within the specified boundaries
    scaled_points = []
    for i in range(dim):
        a, b = boundaries[i]
        scaled_dim = a + sobol_points[:, i] * (b - a)
        scaled_points.append(scaled_dim)

    # Transpose the scaled points to get points per dimension
    scaled_points = np.array(list(map(list, zip(*scaled_points))))
    return scaled_points

In [12]:
iter_points = generate_points(dim,n_point,boundaries[:,0],boundaries[:,1])
iter_points

array([[  0.    ,   0.    ],
       [ 50.    , -50.    ],
       [-50.    ,  50.    ],
       [-25.    , -25.    ],
       [ 75.    ,  75.    ],
       [ 25.    , -75.    ],
       [-75.    ,  25.    ],
       [-62.5   , -37.5   ],
       [ 37.5   ,  62.5   ],
       [ 87.5   , -87.5   ],
       [-12.5   ,  12.5   ],
       [-37.5   , -62.5   ],
       [ 62.5   ,  37.5   ],
       [ 12.5   , -12.5   ],
       [-87.5   ,  87.5   ],
       [-81.25  ,  -6.25  ],
       [ 18.75  ,  93.75  ],
       [ 68.75  , -56.25  ],
       [-31.25  ,  43.75  ],
       [ -6.25  , -81.25  ],
       [ 93.75  ,  18.75  ],
       [ 43.75  , -31.25  ],
       [-56.25  ,  68.75  ],
       [-68.75  , -68.75  ],
       [ 31.25  ,  31.25  ],
       [ 81.25  , -18.75  ],
       [-18.75  ,  81.25  ],
       [-43.75  , -43.75  ],
       [ 56.25  ,  56.25  ],
       [  6.25  , -93.75  ],
       [-93.75  ,   6.25  ],
       [-90.625 , -46.875 ],
       [  9.375 ,  53.125 ],
       [ 59.375 , -96.875 ],
       [-40.62

In [13]:
def encode_number(number, min_value=-10, max_value=10, num_bits=32):
    # Normalize the number to a value between 0 and 1
    normalized = (number - min_value) / (max_value - min_value)
    # Convert it to an integer representation
    int_representation = int(normalized * (2**num_bits - 1))
    # Convert the integer to binary and pad with zeros
    return [int(x) for x in format(int_representation, '0{}b'.format(num_bits))]

def decode_number(genome:Genome, min_value=-10, max_value=10, num_bits=32):
    # Convert the binary string to an integer
    int_representation = int(''.join(map(str, genome)), 2)
    # Scale down to the normalized value
    normalized = int_representation / (2**num_bits - 1)
    # Denormalize to get the real number
    return min_value + normalized * (max_value - min_value)

def encode_list(number_list,min_value=-10, max_value=10, num_bits=32):
    encoded = []
    for number in number_list:
        encoded += encode_number(number,min_value, max_value, num_bits)
    return encoded

def decode_list(encoded_list,min_value=-10, max_value=10, num_bits=32):
    numbers = []
    for i in range(0, len(encoded_list), num_bits):
        binary_list = encoded_list[i:i + num_bits]
        number = decode_number(binary_list, min_value, max_value, num_bits)
        numbers.append(number)
    return numbers


In [14]:
def generate_population(set_of_points:np.ndarray,min_value=-10, max_value=10, num_bits=32) -> Population:
    return [encode_list(set_of_points[point],min_value, max_value, num_bits) for point in range(len(set_of_points))]

def single_point_crossover(a: Genome, b: Genome, print_cutoff=False) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")
    length = len(a)
    # if the length less than 2, then there is no point to do the function
    if length < 2:  
        return a, b
    # generate random number as the cutoff of the crossover
    p = randint(1, length - 1)
    if print_cutoff == True:
        print(p)
    
    return a[0:p] + b[p:], b[0:p] + a[p:]

def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    # num: generate how many chromosome(s) that we want to mutate
    for _ in range(num):
        # index sets which chromosome we want to change
        index = randrange(len(genome))
        # the change algorithm
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome

def fitness_function(genome: Genome, min_value=-10, max_value=10, num_bits=64) -> float:
    X = decode_list(genome, min_value, max_value, num_bits)
    return objective_function(X)

# for convenience, call fitness function from these functions below using partial(...)
def population_fitness(population: Population, fitness_func: FitnessFunc) -> float:
    return sum([fitness_func(genome) for genome in population])

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(gene) for gene in population],
        k=2
    )
def sort_population(population: Population, fitness_func: FitnessFunc, minimize=True) -> Population:
    return sorted(population,key= lambda x: fitness_func(x), reverse=not minimize)

def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))

def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc,binary_mode=False):
    print("GENERATION %02d" % generation_id)
    print("=============")
    if binary_mode == True:
        print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        sorted_population = sort_population(population, fitness_func)
        print(
            "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
        print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                                fitness_func(sorted_population[-1])))
        print("")
    else:
        print(f"Population: {[decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits) for i in range (len(population))]}")
        print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
        sorted_population = sort_population(population, fitness_func)
        print(f"Best: {(decode_list(population[0],min_value=min_value,max_value=max_value,num_bits=num_bits))}")
        print(f"Worst: {(decode_list(population[-1],min_value=min_value,max_value=max_value,num_bits=num_bits))}")
        print("")
    # return sorted_population[0]

def run_evolution(
        populate_func,
        fitness_func,
        fitness_limit: int,
        minimize = False,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        sort_func = sort_population,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    for i in range(generation_limit):
        # print(f"i={i}")
        population = sort_func(population,minimize=minimize)

        if printer is not None:
            printer(population, i, fitness_func)

        if minimize==True:
            cutoff_criteria = fitness_func(population[0])
        else:
            cutoff_criteria = 1-fitness_func(population[0])
        # print(f"cutoff:{cutoff_criteria}")
        # print(decode_list(population[0],min_value=min_value,max_value=max_value,num_bits=64))
        if cutoff_criteria < fitness_limit:
            break

        next_generation = population[0:2]

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population)
            # print(f"j={j}")
            # print(decode_list(parents[0],min_value=min_value,max_value=max_value,num_bits=64),decode_list(parents[1],min_value=min_value,max_value=max_value,num_bits=64))
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i

In [15]:
popA = generate_population(iter_points,num_bits=num_bits,min_value=min_value,max_value=max_value)
print(popA[0])
print(popA[1])
decoded_genome = [decode_list(popA[i],num_bits=num_bits) for i in range (len(popA))]
print(decoded_genome)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0.0, 0.0], [5.0, -5.0], [-5.0, 5.0], [-2.5, -2.5], [7.5, 7.5], [2.5, -7.5], [-7.5, 2.5], [-6.25, -3.75], [3.75, 6.25], [8.75, -8.75], [-1.25, 1.25], [-3.75, -6.25], [6.25, 3.75], [1.25, -1.25], [-8.75, 8.75], [-8.125, -0.625], [

In [16]:
np.array([fitness_function(genome=gene,num_bits=num_bits,min_value=min_value,max_value=max_value) for gene in popA])

array([9.00000000e+00, 1.27503000e+05, 1.22509000e+05, 4.74970000e+04,
       1.28249100e+06, 1.41259000e+05, 7.62470000e+04, 2.30856375e+05,
       5.42256625e+05, 6.77581125e+05, 1.80587500e+03, 5.39450125e+05,
       2.38662875e+05, 2.11237500e+03, 6.62274625e+05, 7.11014062e+03,
       1.15601639e+06, 2.62657016e+05, 3.53117344e+04, 6.19401297e+05,
       9.18367031e+04, 5.73835781e+04, 2.04335172e+05, 9.79577078e+05,
       9.44734219e+04, 6.22077969e+04, 2.90344797e+05, 2.53131766e+05,
       5.43418734e+05, 7.14628141e+05, 9.90310938e+03, 5.05496268e+05,
       2.03515170e+05, 2.11046701e+05, 2.40188867e+03, 5.33865408e+05,
       1.72333529e+05, 2.40327480e+04, 3.20771939e+05, 1.36156560e+06,
       3.18208340e+04, 2.03754428e+05, 2.55914865e+05, 6.02849414e+03,
       1.94173294e+06, 5.54350254e+04, 1.87649588e+05, 6.30669607e+05,
       1.72873080e+05, 9.65042383e+03, 2.67279611e+05, 3.73395293e+04,
       1.33057816e+06, 2.22503145e+04, 4.50543730e+04, 4.00299590e+04,
      

In [17]:
print(selection_pair(population=popA,fitness_func=partial(
    fitness_function,num_bits=num_bits,min_value=min_value,max_value=max_value
)))

[[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [18]:
sorted_population2 = sort_population(popA, fitness_func=partial(
    fitness_function,num_bits=num_bits,min_value=min_value,max_value=max_value
),minimize=True)

for individual in sorted_population2:
    print(fitness_function(individual, num_bits=num_bits,min_value=min_value,max_value=max_value))

print(sorted_population2)


9.0
13.119873046875
1701.760986328125
1805.875
1899.313232421875
2112.375
2401.888671875
6028.494140625
7110.140625
9650.423828125
9903.109375
10881.802734375
16295.931640625
16669.619384765625
22250.314453125
24032.748046875
27044.375244140625
30086.672607421875
30221.658447265625
31820.833984375
35311.734375
37339.529296875
40029.958984375
40262.759033203125
42423.605224609375
45054.373046875
45474.850830078125
47497.0
52358.907958984375
55435.025390625
55960.696044921875
56848.599609375
57383.578125
62207.796875
76247.0
88965.35327148438
91836.703125
93714.90747070312
94473.421875
107057.39428710938
111500.34497070312
122509.0
122972.51293945312
125806.78393554688
127503.0
141259.0
172333.529296875
172873.080078125
187649.587890625
195123.49536132812
196967.26391601562
203515.169921875
203754.427734375
204335.171875
211046.701171875
213423.15942382812
230584.41430664062
230856.375
238662.875
253131.765625
253167.01342773438
255914.865234375
257607.32104492188
262657.015625
267279.61

In [19]:
print_stats(population=popA,generation_id=1,fitness_func=partial(
    fitness_function,num_bits=num_bits,min_value=min_value,max_value=max_value
),binary_mode=False
)

GENERATION 01
Population: [[0.0, 0.0], [50.0, -50.0], [-50.0, 50.0], [-25.0, -25.0], [75.0, 75.0], [25.0, -75.0], [-75.0, 25.0], [-62.5, -37.5], [37.5, 62.5], [87.5, -87.5], [-12.5, 12.5], [-37.5, -62.5], [62.5, 37.5], [12.5, -12.5], [-87.5, 87.5], [-81.25, -6.25], [18.75, 93.75], [68.75, -56.25], [-31.25, 43.75], [-6.25, -81.25], [93.75, 18.75], [43.75, -31.25], [-56.25, 68.75], [-68.75, -68.75], [31.25, 31.25], [81.25, -18.75], [-18.75, 81.25], [-43.75, -43.75], [56.25, 56.25], [6.25, -93.75], [-93.75, 6.25], [-90.625, -46.875], [9.375, 53.125], [59.375, -96.875], [-40.625, 3.125], [-15.625, -71.875], [84.375, 28.125], [34.375, -21.875], [-65.625, 78.125], [-53.125, -84.375], [46.875, 15.625], [96.875, -34.375], [-3.125, 65.625], [-28.125, -9.375], [71.875, 90.625], [21.875, -59.375], [-78.125, 40.625], [-84.375, -53.125], [15.625, 46.875], [65.625, -3.125], [-34.375, 96.875], [-9.375, -28.125], [90.625, 71.875], [40.625, -78.125], [-59.375, 21.875], [-71.875, -15.625], [28.125, 84.3

In [20]:
# print(selection_pair(population=popA,fitness_func=partial(
#     fitness_function,num_bits=num_bits
# )))
population,generation = run_evolution(
    populate_func=partial(
        generate_population,set_of_points = iter_points,num_bits=num_bits,min_value=min_value,max_value=max_value
    ),
    fitness_func= partial(
        fitness_function, num_bits=num_bits,min_value=min_value,max_value=max_value
    ),
    minimize=True,
    sort_func=partial(
        sort_population,fitness_func=partial(
            fitness_function,
            num_bits=num_bits,
            min_value=min_value,max_value=max_value   
        )),
    selection_func=partial(
        selection_pair,
        fitness_func=partial(
            fitness_function,
            num_bits=num_bits,
            min_value=min_value,max_value=max_value)),
    mutation_func=partial(
        mutation,probability=p_mutation
    ),
    fitness_limit=epsilon,
    generation_limit=k_max,
    printer=print_stats
)

GENERATION 00
Population: [[0.0, 0.0], [-1.5625, -1.5625], [10.9375, -14.0625], [-12.5, 12.5], [-14.0625, 10.9375], [12.5, -12.5], [-40.625, 3.125], [-28.125, -9.375], [-81.25, -6.25], [65.625, -3.125], [-93.75, 6.25], [-21.875, 34.375], [53.125, 9.375], [79.6875, 4.6875], [40.625, -78.125], [34.375, -21.875], [92.1875, -7.8125], [7.8125, 26.5625], [29.6875, -45.3125], [46.875, 15.625], [-31.25, 43.75], [-9.375, -28.125], [-71.875, -15.625], [23.4375, 23.4375], [-82.8125, 17.1875], [-59.375, 21.875], [-42.1875, 76.5625], [-25.0, -25.0], [-45.3125, 29.6875], [21.875, -59.375], [57.8125, -23.4375], [3.125, -40.625], [43.75, -31.25], [81.25, -18.75], [-75.0, 25.0], [-95.3125, -20.3125], [93.75, 18.75], [73.4375, -26.5625], [31.25, 31.25], [-32.8125, -32.8125], [-26.5625, 73.4375], [-50.0, 50.0], [48.4375, -51.5625], [-51.5625, 48.4375], [50.0, -50.0], [25.0, -75.0], [84.375, 28.125], [15.625, 46.875], [-78.125, 40.625], [-17.1875, -48.4375], [54.6875, -70.3125], [9.375, 53.125], [96.875, 

Population: [[0.0, 0.0], [-1.5625, -1.5625], [-53.125, -1.5625], [-46.875, 98.4375], [-46.875, 98.4375], [-46.875, 98.4375000000004], [85.9375, 17.1875], [-46.875, 79.6875], [-53.125, 93.7500000007276], [-43.75, 98.4375], [28.125, -90.625], [71.875, -84.375], [-76.5625, 79.68750004656613], [-76.5625, 79.68750004656613], [9.375, 71.875], [75.0, -84.375], [4.6875, -81.25], [75.0, -90.625], [75.0, -95.3125], [-76.5625, 96.87504768371582], [-76.5625, 98.4375], [-76.5625, 98.4375], [78.125, -90.625], [-20.3125, 98.4375], [-20.3125, 98.4375], [85.9375, -89.0625], [4.6875, -90.625], [-89.0625, 85.93750009313226], [85.9375, -90.62347412109375], [85.9375, -90.625], [-53.1249999998181, -64.06249997671694], [98.4375, -76.46484374999996], [90.625, -89.0625], [98.4375, -84.375], [98.43750596046448, -84.375], [96.875, -89.0625], [98.4375, -90.625], [98.4619140625, -90.625], [-43.75, -76.5625], [-46.875, -76.46484375], [71.875, 71.875], [-43.75, -84.375], [-20.3125, -95.3125], [-20.3125, -95.3125], [

In [21]:
decode_list(population[0],min_value=min_value,max_value=max_value,num_bits=num_bits)

[0.0, 0.0]

In [22]:
for i in range (len(population)):
    print(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits))
    print(objective_function(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits)))

[0.0, 0.0]
9.0
[-1.5625, -1.5625]
13.119873046875
[84.375, 98.4375953914423]
2611572.115017354
[85.9375, 98.4375]
2642533.266845703
[84.37576293945312, 98.4375953907147]
2611587.233272974
[99.21875000072762, 98.4375]
2906147.9214011803
[84.375, 98.4375]
2611566.164794922
[84.375, 98.43788146972656]
2611589.9597444516
[99.2187500007276, 98.43759536743164]
2906154.428938342
[98.43750000000284, 98.43826296273619]
2890673.4725921247
[85.9375000007276, 98.43750004656613]
2642533.269793496
[98.4375, 98.43826293945312]
2890673.471010478
[75.0000000007276, 98.43826293945312]
2426013.397180808
[96.87500000000009, 98.4375]
2859576.4187011737
[75.0000000003638, 95.31326298601925]
2246966.804079189
[97.65625000291038, 98.43788146972656]
2875123.6064940086
[84.375, 99.21951293945312]
2660629.9458770906
[71.875, 98.43750000000017]
2364180.9108886817
[98.4375, 98.4375]
2890621.645751953
[86.3281250007276, 98.43826293945312]
2650324.74651146
[85.9375, 98.43759536815924]
2642539.2744386196
[96.875, 98.

In [23]:
ss = sorted(population,key=lambda x:fitness_function(x,min_value=min_value,max_value=max_value,num_bits=num_bits))
for i in range (len(ss)):
    print(decode_list(ss[i],min_value=min_value,max_value=max_value,num_bits=num_bits))
    print(objective_function(decode_list(ss[i],min_value=min_value,max_value=max_value,num_bits=num_bits)))

[0.0, 0.0]
9.0
[-1.5625, -1.5625]
13.119873046875
[71.875, 95.31326298601925]
2188972.118244091
[75.0000000003638, 95.31326298601925]
2246966.804079189
[71.875, 98.4375]
2364180.910888672
[71.875, 98.4375]
2364180.910888672
[71.87500000000034, 98.4375]
2364180.9108886784
[71.875, 98.43750000000017]
2364180.9108886817
[71.875, 98.43759536815924]
2364186.3890762893
[71.875, 98.43826298601925]
2364224.7389537804
[71.97265625, 98.4375]
2366111.185852051
[73.4375, 98.43831062316895]
2395116.9524254063
[73.4375, 98.43831086764112]
2395116.96661946
[75.0, 98.4375]
2425968.6306152344
[75.00000000072768, 98.4375953907147]
2425974.227786334
[75.0, 98.43788171419874]
2425991.0281707747
[75.0, 98.43826293945312]
2426013.397166417
[75.0000000007276, 98.43826293945312]
2426013.397180808
[84.375, 98.4375]
2611566.164794922
[84.375, 98.4375]
2611566.164794922
[84.375, 98.4375]
2611566.164794922
[84.375, 98.43750000000017]
2611566.1647949326
[84.3750000007276, 98.4375]
2611566.1648093397
[84.3750000058

In [24]:
socores = [fitness_function(population[i],-100,100,num_bits=num_bits) for i in range(len(population))]
asort = [x for _,x in sorted(zip(socores,population))]
bsort = sorted(socores)
for i in range (len(asort)):
    print(bsort[i])
    print(decode_list(asort[i],num_bits=num_bits))


9.0
[0.0, 0.0]
13.119873046875
[-0.15625, -0.15625]
2188972.118244091
[7.1875, 9.531326298601925]
2246966.804079189
[7.50000000003638, 9.531326298601925]
2364180.910888672
[7.1875, 9.84375]
2364180.910888672
[7.1875, 9.84375]
2364180.9108886784
[7.1875000000000355, 9.84375]
2364180.9108886817
[7.1875, 9.843750000000018]
2364186.3890762893
[7.1875, 9.843759536815924]
2364224.7389537804
[7.1875, 9.843826298601925]
2366111.185852051
[7.197265625, 9.84375]
2395116.9524254063
[7.34375, 9.843831062316895]
2395116.96661946
[7.34375, 9.843831086764112]
2425968.6306152344
[7.5, 9.84375]
2425974.227786334
[7.500000000072767, 9.84375953907147]
2425991.0281707747
[7.5, 9.843788171419874]
2426013.397166417
[7.5, 9.843826293945312]
2426013.397180808
[7.50000000007276, 9.843826293945312]
2611566.164794922
[8.4375, 9.84375]
2611566.164794922
[8.4375, 9.84375]
2611566.164794922
[8.4375, 9.84375]
2611566.1647949326
[8.4375, 9.843750000000018]
2611566.1648093397
[8.43750000007276, 9.84375]
2611566.167814

In [25]:
socores = [objective_function(decode_list(population[i],min_value=min_value,max_value=max_value,num_bits=num_bits)) for i in range(len(population))]
asort = [x for _,x in sorted(zip(socores,population))]
bsort = sorted(socores)
for i in range (len(asort)):
    print(bsort[i])
    print(decode_list(asort[i],num_bits=num_bits))


9.0
[0.0, 0.0]
13.119873046875
[-0.15625, -0.15625]
2188972.118244091
[7.1875, 9.531326298601925]
2246966.804079189
[7.50000000003638, 9.531326298601925]
2364180.910888672
[7.1875, 9.84375]
2364180.910888672
[7.1875, 9.84375]
2364180.9108886784
[7.1875000000000355, 9.84375]
2364180.9108886817
[7.1875, 9.843750000000018]
2364186.3890762893
[7.1875, 9.843759536815924]
2364224.7389537804
[7.1875, 9.843826298601925]
2366111.185852051
[7.197265625, 9.84375]
2395116.9524254063
[7.34375, 9.843831062316895]
2395116.96661946
[7.34375, 9.843831086764112]
2425968.6306152344
[7.5, 9.84375]
2425974.227786334
[7.500000000072767, 9.84375953907147]
2425991.0281707747
[7.5, 9.843788171419874]
2426013.397166417
[7.5, 9.843826293945312]
2426013.397180808
[7.50000000007276, 9.843826293945312]
2611566.164794922
[8.4375, 9.84375]
2611566.164794922
[8.4375, 9.84375]
2611566.164794922
[8.4375, 9.84375]
2611566.1647949326
[8.4375, 9.843750000000018]
2611566.1648093397
[8.43750000007276, 9.84375]
2611566.167814

In [26]:
# import random

# # Objective function
# def objective_function(x):
#     return x ** 2

# # Create initial population
# def create_population(pop_size, min_val, max_val):
#     return [random.uniform(min_val, max_val) for _ in range(pop_size)]

# # Crossover between two parents to create two children
# def crossover(parent1, parent2):
#     alpha = random.random()
#     child1 = alpha * parent1 + (1 - alpha) * parent2
#     child2 = alpha * parent2 + (1 - alpha) * parent1
#     return child1, child2

# # Mutation
# def mutate(x, mutation_rate, min_val, max_val):
#     if random.random() < mutation_rate:
#         return random.uniform(min_val, max_val)
#     return x

# # Genetic algorithm
# def genetic_algorithm(objective, pop_size, generations, mutation_rate, min_val, max_val):
#     population = create_population(pop_size, min_val, max_val)
#     best_solution = None
#     best_score = float('-inf')

#     for _ in range(generations):
#         # Evaluate all solutions in the population
#         scores = [objective(x) for x in population]
        
#         # Check for new best solution
#         for i in range(pop_size):
#             if scores[i] > best_score:
#                 best_solution, best_score = population[i], scores[i]

#         # Select parents and create next generation
#         sorted_population = [x for _, x in sorted(zip(scores, population), reverse=True)]
#         # print(sorted_population)
#         next_generation = []
#         for i in range(0, pop_size, 2):
#             parent1, parent2 = sorted_population[i], sorted_population[i+1]
#             child1, child2 = crossover(parent1, parent2)
#             next_generation.extend([mutate(child1, mutation_rate, min_val, max_val),
#                                     mutate(child2, mutation_rate, min_val, max_val)])

#         population = next_generation

#     return best_solution, best_score

# # Parameters
# pop_size = 100
# generations = 100
# mutation_rate = 0.01
# min_val = -10
# max_val = 10

# # Run Genetic Algorithm
# best_solution, best_score = genetic_algorithm(objective_function, pop_size, generations, mutation_rate, min_val, max_val)
# print(f"Best Solution: {best_solution}, Score: {best_score}")
